In [1]:
from keras.applications import VGG19


Using TensorFlow backend.


In [2]:
model=VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [3]:
for layers in model.layers:
    layers.trainable=False

In [4]:
model.layers[5].trainable

False

In [5]:
last_layer=model.output
last_layer

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

In [6]:
from keras.layers import Dense

In [7]:
from keras.models import Sequential

In [8]:
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D

In [9]:
last_layer=GlobalAveragePooling2D()(last_layer)

In [10]:
last_layer=Dense(512,activation='relu')(last_layer)

In [11]:
last_layer=Dense(250,activation='relu')(last_layer)

In [12]:
last_layer=Dense(100,activation='relu')(last_layer)

In [13]:
last_layer=Dense(50,activation='relu')(last_layer)

In [14]:
last_layer=Dense(10,activation='relu')(last_layer)

In [15]:
last_layer=Dense(2,activation='sigmoid')(last_layer)

In [16]:
from keras.models import Model

In [17]:
new_model=Model(inputs=model.input , outputs=last_layer)

In [18]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
from keras_preprocessing.image import ImageDataGenerator

In [20]:
train_data_dir = 'F:/MLOPS/practise/ENVIRONMENT2/ankush_dataset_images/train/'
validation_data_dir = 'F:/MLOPS/practise/ENVIRONMENT2/ankush_dataset_images/test/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical')

Found 2398 images belonging to 2 classes.
Found 72 images belonging to 2 classes.


In [21]:
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("face_detection.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
new_model.compile(loss = 'binary_crossentropy',
              optimizer = Adam(lr = 0.0001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 2398
nb_validation_samples = 72

# We only train 5 EPOCHS 
epochs = 3
batch_size = 32

history = new_model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/3
74/74 [==============================] - 653s 9s/step - loss: 0.5877 - accuracy: 0.7263 - val_loss: 0.6098 - val_accuracy: 0.7812

Epoch 00001: val_loss improved from inf to 0.60982, saving model to face_detection.h5
Epoch 2/3
74/74 [==============================] - 647s 9s/step - loss: 0.2950 - accuracy: 0.9577 - val_loss: 0.3618 - val_accuracy: 0.8625

Epoch 00002: val_loss improved from 0.60982 to 0.36175, saving model to face_detection.h5
Epoch 3/3
74/74 [==============================] - 656s 9s/step - loss: 0.1197 - accuracy: 0.9791 - val_loss: 0.1681 - val_accuracy: 0.8250

Epoch 00003: val_loss improved from 0.36175 to 0.16814, saving model to face_detection.h5


In [22]:
train_generator.class_indices

{'ANKUSH': 0, 'ARTI': 1}